# General

In [48]:

# imports

import os

import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit


In [ ]:
def get_data(file_path, cut = [200,800]):
    '''
    take raw data (.dat) and transform it to pd data frame 
    '''
    df = pd.read_csv(file_path, delimiter = '\t', names = ['Wavelength', 'Intensity'], header = 0 )
    return df[(cut[0] < df['Wavelength']) & (df['Wavelength'] < cut[1])]

def plot_glow(data, y = 'Intensity', title = "Graph", to_show = True, label = '', error = False):
    '''
    plot data and fit 
    '''
    # plot data points
    # data = data.groupby(['Temperature (K)'], as_index=False).mean()
    x_data, y_data = data['Wavelength'], data[y]  # call data
    if error:
        x_err, y_err = data['Wavelength err'], data[y + ' err']
        plt.errorbar(x_data, y_data,x_err,y_err, ',', label = label)
    else:
        plt.plot(x_data, y_data, '.', label = label)
    
    plt.legend(loc='best')
    plt.grid()
    plt.xlabel('Wave length [nm]')
    # plt.ylabel(r'$\rho$ ' +' [V]')
    plt.ylabel(y +' [V]')

    plt.title(title)
    plt.show(to_show)

def culc_absorbance(I, I0,A = None):
    '''
    culculate absorbance dataframe from intensity
    '''
    A = pd.DataFrame()
    A['Absorbance'] = -np.log(I['Intensity']/I0['Intensity'])
    A['Wavelength'] = I['Wavelength']

    return A

def get_concentration(file_name):
    '''
    get concentration from file name
    '''
    return float(file_name.split('_')[0]) # file name is like 0.1_M.dat

def culc_molar_absorption_coefficient(slope, error, l = 1):
    '''
    culculate molar absorption coefficient from absorbance and concentration
    '''
    return slope / l, error / l

def linear_func(x,a0,a1): # define the modle fucntion - linear
    return a1 * x + a0

def linear_fit(x,y):
    '''
    fit linear function
    '''
    popt, pcov = curve_fit(linear_func, x, y)
    return popt, np.sqrt(np.diag(pcov))




# Part 1:  Beer-Lambert's Law Validation

In [ ]:
# define folder
mes_folder_1 = r"C:\Users\yaniv\OneDrive\Desktop\yehonathan\לימודים\אוניברסיטה\2024-2025\מעבדה בכימיה פיזיקלית 2\קינטיקה כימית\part 1"
I0_1 = get_data(mes_folder_1 + r"\I0.dat")

# get absobance data frame
Absorbabce_all = pd.DataFrame()
Absorbabce_all['Wavelength'] = I0_1['Wavelength']
for file in os.listdir(mes_folder_1):
    if file.endswith(".dat") and file != 'I0.dat':
        data = get_data(mes_folder_1 + '\\' + file)
        C = get_concentration(file)
        Absorbabce_all[str(C) + '_M'] = culc_absorbance(data, I0)['Absorbance']

display(Absorbabce_all)

In [ ]:
# Create a dictionary to hold the data for each wavelength
data_dict = {'Concentration': [get_concentration(file) for file in os.listdir(mes_folder_1) if file.endswith(".dat") and file != 'I0.dat']}
for wavelength in Absorbabce_all['Wavelength']:
    data_dict[str(wavelength)] = Absorbabce_all[Absorbabce_all['Wavelength'] == wavelength].drop(columns='Wavelength').values[0]

# Use pd.concat to join all columns at once
Absorbabce_all2 = pd.concat([pd.DataFrame(data_dict)], axis=1)

display(Absorbabce_all2)

In [ ]:
wavelengths_list = [350.374, 352.267]
l = 1 # cm
molar_absorption = {'wavelength [nm]': wavelengths_list, 'molar_absorption': [], 'molar_absorption_err': []}
for wavelength in wavelengths_list:
    # Fit a linear function to the data
    fit = linear_fit(Absorbabce_all2['Concentration'], Absorbabce_all2[str(wavelength)])
    plt.plot(Absorbabce_all2['Concentration'], Absorbabce_all2[str(wavelength)], '.', label=str(wavelength) + ' nm')
    plt.plot(Absorbabce_all2['Concentration'], linear_func(Absorbabce_all2['Concentration'], *fit[0]), label=str(wavelength) + ' nm fit')

    # Calculate molar absorption coefficient
    epsilon = culc_molar_absorption_coefficient(fit[0][1], fit[1][1], l)
    molar_absorption['molar_absorption'].append(epsilon[0])
    molar_absorption['molar_absorption_err'].append(epsilon[1]) 

plt.legend()
plt.xlabel('Concentration [M]')
plt.ylabel('Absorbance')
plt.title('Absorbance Vs. Concentration', fontsize=20)
plt.show()

molar_absorption_df = pd.DataFrame(molar_absorption)
display(molar_absorption_df)

# Part 2: Determination of Optimal Acid:Acetone:Iodide Ratio


# Part 3: Finding the Reaction Order for Each Component

In [ ]:
# define folder
mes_folder_3 = r"C:\Users\yaniv\OneDrive\Desktop\yehonathan\לימודים\אוניברסיטה\2024-2025\מעבדה בכימיה פיזיקלית 2\קינטיקה כימית\part 3"
I0_3 = get_data(mes_folder_3 + r"\I0.dat")

# get absobance data frame
Absorbabce_all = pd.DataFrame()
Absorbabce_all['Wavelength'] = I0_3['Wavelength']
for file in os.listdir(mes_folder_3):
    if file.endswith(".dat") and file != 'I0.dat':
        data = get_data(mes_folder_3 + '\\' + file)
        C = get_concentration(file)
        Absorbabce_all[str(C) + '_M'] = culc_absorbance(data, I0_3)['Absorbance']

display(Absorbabce_all)